In [1]:
# coding='utf-8'
import os
import sys
import numpy as np
import time
import datetime
import json
import importlib
import logging
import shutil
import cv2
import random

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import NullLocator

import torch
import torch.nn as nn

from yolo_model import yoloModel
from PASCAL_Dataloader import create_split_loaders
from YOLO_Loss import YoloLoss
from utils import NMS

cmap = plt.get_cmap('tab20b')
colors = [cmap(i) for i in np.linspace(0, 1, 20)]

In [2]:
def test(config):
    
    # Load and initialize network
    net = yoloModel(config)

    # Set data parallel
    net = net.cuda()

    # Restore pretrain model
    if config["pretrain_snapshot"]:
        logging.info("load checkpoint from {}".format(config["pretrain_snapshot"]))
        state_dict = torch.load(config["pretrain_snapshot"])
        net.load_state_dict(state_dict)
    else:
        raise Exception("missing pretrain_snapshot!!!")

    # YOLO loss with 3 scales
    yolo_losses = []
    for i in range(3):
        yolo_losses.append(YoloLoss(config["classes"], (config["img_w"], config["img_h"]), config["anchors"][i]))

    
    # prepare images path
    images_name = os.listdir(config["images_path"])
    images_path = [os.path.join(config["images_path"], name) for name in images_name]
    if len(images_path) == 0:
        raise Exception("no image found in {}".format(config["images_path"]))
    '''
    root_dir = os.getcwd()
    train_loader, val_loader, test_loader = create_split_loaders(root_dir, config['batch_size'])
    '''

    # Start inference
    batch_size = config["batch_size"]
    for n in range(0, len(images_path), batch_size):
        
        # preprocess
        images = []
        images_origin = []
        for path in images_path[n*batch_size: (n+1)*batch_size]:
            logging.info("processing: {}".format(path))
            image = cv2.imread(path, cv2.IMREAD_COLOR)
            if image is None:
                logging.error("read path error: {}. skip it.".format(path))
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            images_origin.append(image)  # keep for save result
            image = cv2.resize(image, (config["img_w"], config["img_h"]),
                               interpolation=cv2.INTER_LINEAR)
            image = image.astype(np.float32)
            image /= 255.0
            image = np.transpose(image, (2, 0, 1))
            image = image.astype(np.float32)
            images.append(image)
        
        #images, labels = sample["image"], sample["label"]
        images = np.asarray(images)
        images = torch.from_numpy(images).cuda()
        # inference
        with torch.no_grad():
            outputs = net(images)
            output_list = []
            for i in range(3):
                output_list.append(yolo_losses[i](outputs[i]))
            print(output_list)
            output = torch.cat(output_list, 1)
            batch_detections = NMS(output, config["classes"])

        # write result images. Draw bounding boxes and labels of detections
        classes = open(config["classes_names_path"], "r").read().split("\n")[:-1]
        if not os.path.isdir("./output/"):
            os.makedirs("./output/")
        for idx, detections in enumerate(batch_detections):
            plt.figure()
            fig, ax = plt.subplots(1)
            ax.imshow(images_origin[idx])
            if detections is not None:
                unique_labels = detections[:, -1].cpu().unique()
                n_cls_preds = len(unique_labels)
                bbox_colors = random.sample(colors, n_cls_preds)
                for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:
                    color = bbox_colors[int(np.where(unique_labels == int(cls_pred))[0])]
                    
                    # Rescale coordinates to original dimensions
                    ori_h, ori_w = images[idx].shape[:2]
                    pre_h, pre_w = config["img_h"], config["img_w"]
                    box_h = ((y2 - y1) / pre_h) * ori_h
                    box_w = ((x2 - x1) / pre_w) * ori_w
                    y1 = (y1 / pre_h) * ori_h
                    x1 = (x1 / pre_w) * ori_w
                    
                    # Create a Rectangle patch
                    bbox = patches.Rectangle((x1, y1), box_w, box_h, linewidth=2,
                                             edgecolor=color,
                                             facecolor='none')
                    # Add the bbox to the plot
                    ax.add_patch(bbox)
                    # Add label
                    plt.text(x1, y1, s=classes[int(cls_pred)], color='white',
                             verticalalignment='top',
                             bbox={'color': color, 'pad': 0})
            # Save generated image with detections
            plt.axis('off')
            plt.gca().xaxis.set_major_locator(NullLocator())
            plt.gca().yaxis.set_major_locator(NullLocator())
            plt.savefig('output/{}_{}.jpg'.format(n, idx), bbox_inches='tight', pad_inches=0.0)
            plt.close()
    logging.info("Save all results to ./output/")  

In [3]:
def main():
    logging.basicConfig(level=logging.DEBUG,
                        format="[%(asctime)s %(filename)s] %(message)s")

    config = {}
    config["batch_size"] = 1
    config['backbone_name'] = "darknet_53"
    config['backbone_pretrained'] = ""
    config['anchors'] = [[[116, 90], [156, 198], [373, 326]],
                                [[30, 61], [62, 45], [59, 119]],
                                [[10, 13], [16, 30], [33, 23]]]
    config['classes'] = 20
    config['img_h'] = config['img_w'] = 416
    config['confidence_threshold'] = 0.5
    config['pretrain_snapshot'] = "official_yolov3_weights_pytorch.pth"
    config['classes_names_path'] = "./data/voc.names"
    config['images_path'] = "./images"

    # Start training
    test(config)

In [4]:
if __name__ == "__main__":
    main()

[2019-05-30 14:49:04,890 <ipython-input-2-546188dbd91d>] load checkpoint from official_yolov3_weights_pytorch.pth


RuntimeError: Error(s) in loading state_dict for yoloModel:
	Missing key(s) in state_dict: "backbone.conv1.weight", "backbone.bn1.weight", "backbone.bn1.bias", "backbone.bn1.running_mean", "backbone.bn1.running_var", "backbone.layer1.ds_conv.weight", "backbone.layer1.ds_bn.weight", "backbone.layer1.ds_bn.bias", "backbone.layer1.ds_bn.running_mean", "backbone.layer1.ds_bn.running_var", "backbone.layer1.residual_0.conv1.weight", "backbone.layer1.residual_0.bn1.weight", "backbone.layer1.residual_0.bn1.bias", "backbone.layer1.residual_0.bn1.running_mean", "backbone.layer1.residual_0.bn1.running_var", "backbone.layer1.residual_0.conv2.weight", "backbone.layer1.residual_0.bn2.weight", "backbone.layer1.residual_0.bn2.bias", "backbone.layer1.residual_0.bn2.running_mean", "backbone.layer1.residual_0.bn2.running_var", "backbone.layer2.ds_conv.weight", "backbone.layer2.ds_bn.weight", "backbone.layer2.ds_bn.bias", "backbone.layer2.ds_bn.running_mean", "backbone.layer2.ds_bn.running_var", "backbone